In [1]:
import numpy as np 
import pandas as pd 
import uncertainties
from uncertainties import ufloat_fromstr
# from uncertainties.umath import
from uncertainties import unumpy

In [2]:
# --- function definition --- 

def current(V, R):
    return (V/R)

def prob_scat(IP1, IP2, IS1, IS2): 
    return 1 - (IP1*IS2)/(IP2*IS1)

def mean_free(P):
    U = 1 - P 
    return -0.7/(unumpy.log(U))

def prob_col(IP1, IP2, IS1, IS2):
    p = 1e-3 #Torr 
    l = 0.7 #cm 
    return (-p/l)*(unumpy.log((IP1*IS2)/(IP2*IS1)))

In [3]:
# --- data import --- 
voltage1, plate_voltage1, shield_voltage1  = np.loadtxt("forward_bias.dat", unpack = True) 
voltage2, plate_voltage2, shield_voltage2  = np.loadtxt("forward_bias_cooled.dat", unpack = True) 
voltage3, shield_voltage3  = np.loadtxt("reverse_bias_cooled.dat", unpack = True) 


# --- intial data manipulation --- 

IP1 = current(plate_voltage1, 10000)
IS1 = current(shield_voltage1, 100)

IP2 = current(plate_voltage2, 10000)
IS2 = current(shield_voltage2, 100)

P = prob_scat(IP1, IP2, IS1, IS2)
l = 0.7 #cm 
lam = mean_free(P)
print(np.shape(lam))

# -- Reverse Polarity --- 

IS3 = current(shield_voltage3, 100)
# print(shield_voltage3)
# print(IS3)
LnIS3 = np.log(IS3)

(45,)


In [10]:
# --- uncertaintiy def --- 

# --- Voltage Uncertainty --- 
vuc1 =  unumpy.uarray(voltage1, voltage1*0.008)
vuc2 =  unumpy.uarray(voltage2, voltage2*0.008)
vuc3 =  unumpy.uarray(voltage3, voltage3*0.008)

# --- Plate Voltages uncertainty from manual \pm 0.08% --- 
pv1u = unumpy.uarray(plate_voltage1, plate_voltage1*0.008)
pv2u = unumpy.uarray(plate_voltage2, plate_voltage2*0.008)

# --- Shield Voltages uncertainty from manual \pm 0.08% --- 
sv1u = unumpy.uarray(shield_voltage1, shield_voltage1*0.008)
sv2u = unumpy.uarray(shield_voltage2, shield_voltage2*0.008)
sv3u = unumpy.uarray(shield_voltage3, shield_voltage3*0.008)

[0.0+/-0 0.1+/-0.0008 0.2+/-0.0016 0.3+/-0.0024 0.4+/-0.0032 0.5+/-0.004
 0.6+/-0.0048 0.7+/-0.0056 0.8+/-0.0064 0.9+/-0.007200000000000001
 1.0+/-0.008 1.1+/-0.0088 1.2+/-0.0096 1.3+/-0.010400000000000001
 1.4+/-0.0112 1.5+/-0.012 1.6+/-0.0128 1.7+/-0.0136
 1.8+/-0.014400000000000001 1.9+/-0.0152 2.0+/-0.016 2.5+/-0.02
 3.0+/-0.024 3.5+/-0.028 4.0+/-0.032 4.5+/-0.036000000000000004 5.0+/-0.04
 5.5+/-0.044 6.0+/-0.048 6.5+/-0.052000000000000005 7.0+/-0.056 7.5+/-0.06
 8.0+/-0.064 8.5+/-0.068 9.0+/-0.07200000000000001 9.5+/-0.076 10.0+/-0.08
 10.5+/-0.084 11.0+/-0.088 11.5+/-0.092 12.0+/-0.096 12.5+/-0.1
 13.0+/-0.10400000000000001 13.5+/-0.108 14.0+/-0.112]


In [5]:
# --- Current Uncertainty --- 

IPU1 = current(pv1u, 10000)
ISU1 = current(sv1u, 100)

IPU2 = current(pv2u, 10000)
ISU2 = current(sv2u, 100)

ISU3 = current(sv3u, 100)

# print(IPU1)

In [6]:
# --- Uncer on collisions --- 

PUC = prob_scat(IPU1, IPU2, ISU1, ISU2)
CUC = prob_col(IPU1, IPU2, ISU1, ISU2)
lamUC = mean_free(PUC)
LnIS3UC = unumpy.log(ISU3)

In [7]:
# --- Contact Voltage --- 
CV = ufloat_fromstr("0.196+/-0.021")

# --- Thermonic Emmission Uncertainty --- 
slope1 = ufloat_fromstr("9.6+/-0.163") # given by cov matrix 
slope2 = ufloat_fromstr("11.16+/-0.211")

sluc = slope1 + slope2
print(sluc)
print(sluc.s)

TEUC = ufloat_fromstr("0.156+/-0.267")

20.76+/-0.27
0.26662708039507166


In [8]:
# --- Corrected Voltages Uncertainty --- 

v_corr_uc = vuc1 - sv1u +  pv1u + TEUC + CV 
Corrected_Voltage_Uncer = unumpy.std_devs(v_corr_uc)    

In [9]:
# --- Output to CSV. --- 

prob_uncer = unumpy.std_devs(PUC)
col_uncer = unumpy.std_devs(CUC)
freepath_uncer = unumpy.std_devs(lamUC)
shieldcurrent_log_uncer = unumpy.std_devs(LnIS3UC)

Output = [prob_uncer, col_uncer, freepath_uncer, Corrected_Voltage_Uncer]
print(Output)
dataset = pd.DataFrame({'Uncertainty on Probablity of Scattering': Output[0], 'Uncertainty on Probablity of Collisions': Output[1], 'Uncertainty on Mean Path Length': Output[2], 'Uncertainty on Corrected Voltage': Output[3]})
# print(dataset)

dataset.to_csv('uncer.csv')

[array([0.00496552, 0.00585366, 0.00873931, 0.01154098, 0.01308887,
       0.01402777, 0.01454795, 0.01406742, 0.01364211, 0.01301847,
       0.01238132, 0.01159401, 0.01063183, 0.00983908, 0.00903948,
       0.00830934, 0.00747844, 0.00682165, 0.00657282, 0.00643699,
       0.0058663 , 0.00350067, 0.00258743, 0.00197515, 0.00161519,
       0.00135637, 0.00119064, 0.00106312, 0.000984  , 0.00096076,
       0.00095549, 0.00096728, 0.00097586, 0.00100169, 0.00104171,
       0.00110945, 0.00122133, 0.0013962 , 0.00163042, 0.00204019,
       0.00175198, 0.00244771, 0.01521526, 0.01593832, 0.01642632]), array([2.28571429e-05, 2.28571429e-05, 2.28571429e-05, 2.28571429e-05,
       2.28571429e-05, 2.28571429e-05, 2.28571429e-05, 2.28571429e-05,
       2.28571429e-05, 2.28571429e-05, 2.28571429e-05, 2.28571429e-05,
       2.28571429e-05, 2.28571429e-05, 2.28571429e-05, 2.28571429e-05,
       2.28571429e-05, 2.28571429e-05, 2.28571429e-05, 2.28571429e-05,
       2.28571429e-05, 2.28571429e-05, 